In [ ]:
import numpy as np
import time as time

In [ ]:
class Agent:
  def __init__(self):
    self.weights = np.zeros(6)
    self.s = np.ones(6)
    self.t = 1

  def get_action(self,runs_scored,Q_no=1):
    action = self.kl_ucb()
    self.t+=1
    # print(action, self.weights)
    return action

  def get_KL(self,w,q):
    if w==0:
      KL = np.log(1/(1-q))
    else:
      KL = w*np.log(w/q) + (1-w)*np.log((1-w)/(1-q))
    return KL

  def get_kl_ucb(self):
    # return self.weights + np.sqrt(1.5*np.log(self.t)/self.s)
    UCB = []
    for i in range(6):
      w = self.weights[i]
      C = (np.log(self.t) + 3*np.log(np.log(self.t)))/self.s[i]
      q = minimize_scalar(lambda q: (self.get_KL(w,q) - C)**2,bounds=(w,1)).x
      UCB.append(min(q,1))
    return UCB

  def get_kl_lcb(self):
    # return self.weights - np.sqrt(1.5*np.log(self.t)/self.s)
    UCB = []
    for i in range(6):
      w = self.weights[i]
      C = (np.log(self.t) + 3*np.log(np.log(self.t)))/self.s[i]
      q = minimize_scalar(lambda q: (self.get_KL(w,q) - C)**2,bounds=(0,w)).x
      UCB.append(max(q,0))
    return UCB

  def kl_ucb(self):
    if self.t<7:
      return self.t-1
    # return np.random.randint(0,6)
    KL_UCB = self.get_kl_ucb()
    a_t = np.argmax(KL_UCB)
    return a_t

  def kl_lcb(self):
    if self.t<7:
      return self.t-1
    # return np.random.randint(0,6)
    KL_UCB = self.get_kl_lcb()
    a_t = np.argmin(KL_UCB)
    return a_t





In [ ]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__p_out =np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_run =np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__wickets_left = 4
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__batting_order = np.array([0,1,2,3,4])

  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket, self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun = self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__wickets_left = 4
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      if (self.__wickets_left > 0 ) :
        action = self.__get_action()
        self.__wicket, self.__runs_scored   = self.__get_outcome(action)
        self.__total_runs     = self.__total_runs + self.__runs_scored
        if (self.__wicket > 0 ):
          self.__wickets_left = self.__wickets_left -1
        self.__total_wickets  = self.__total_wickets + self.__wicket
        if (self.__wickets_left == 0):
          self.__get_action()
    return self.__total_runs, self.__total_wickets, self.__run_time


In [ ]:
num_matches = 10
num_balls = 60
agent = Agent(num_balls,num_matches)
environment = Environment(100,agent)
score = np.zeros((num_matches,1))
run_time = np.zeros((num_matches,1))
wicket = np.zeros((num_matches,1))
for i in range(num_matches):
  score[i],wicket[i],run_time[i] = environment.innings()
